# Tumour Growth Model in Absense of Treatment with a Constant Gaussian Error

The tumour growth inhibitory effects of Erlotinib and Gefitinib were modelled with a population PKPD model in [1]. A population PKPD model is a hierarchical model which consists of a structural model, a population model, and an error model. Each sub-model captures a different aspect of the tumour growth inhibition biology, and is parametrised by a set of parameters.

We introduced the structural model proposed in [1] in [n1] and guranteed that the model parameters are identifiable by transforming the search space [n2].

In this notebook we model the error of the structural lung cancer growth model in absence of treatment. In particular we will choose a Gaussian error of the model output that does not depend on the tumour volume.

## Bibliography

- <a name="ref1"> [1] </a> Eigenmann et. al., Combining Nonclinical Experiments with Translational PKPD Modeling to Differentiate Erlotinib and Gefitinib, Mol Cancer Ther (2016)

## Notebook references

- <a name="ref1"> [n1] </a> [Structural Lung Cancer Growth Model in Absence of Treatment](https://nbviewer.jupyter.org/github/DavAug/ErlotinibGefitinib/blob/master/notebooks/lung_cancer/control_growth/structural_model.ipynb)
- <a name="ref1"> [n2] </a> [Identifiability of Structural Model for Lung Cancer Growth in Absence of Treatment](https://nbviewer.jupyter.org/github/DavAug/ErlotinibGefitinib/blob/master/notebooks/lung_cancer/control_growth/identifiability_structural_model.ipynb)

[Back to project overview](https://github.com/DavAug/ErlotinibGefitinib/blob/master/README.md) | [Back to lung cancer control growth overview](https://nbviewer.jupyter.org/github/DavAug/ErlotinibGefitinib/blob/master/notebooks/lung_cancer/control_growth/overview.ipynb) | [Back to lung cancer control growth error model overview](https://nbviewer.jupyter.org/github/DavAug/ErlotinibGefitinib/blob/master/notebooks/lung_cancer/control_growth/error_model.ipynb)